<h1 style="font-family:Corbel Light; color:#97a9fd">Projet 8 : Déployez un modèle dans le cloud </h1>

<h2 style="font-family:Corbel Light; color:#737373">Démarrage de la session spark</h2>

In [ ]:
# L'exécution de cette cellule démarre l'application Spark

Affichage des informations sur la session en cours et liens vers Spark UI :

In [ ]:
%%info

<h2 style="font-family:Corbel Light; color:#737373">Import des librairies</h2>

In [ ]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

from pyspark.ml.feature import PCA
from pyspark.ml.functions import array_to_vector, vector_to_array

<h2 style="font-family:Corbel Light; color:#737373">Traitement des données</h2>

Emplacement sur S3 du bucket dans lequel stocker les données (input et output):

In [ ]:
PATH = 's3://projet8-oc-082023-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

<h3 style="font-family:Corbel Light; color:#737373">Chargement des données</h3>

In [ ]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [ ]:
images.show(5)

On ne conserve que l'emplacement de l'image ("path") et on ajoute une colonne contenant les **labels** (i.e. la variété du fruit ou du légume représenté sur l'image):

In [ ]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

<h3 style="font-family:Corbel Light; color:#737373">Préparation du modèle</h3>

In [ ]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [ ]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [ ]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [ ]:
new_model.summary()

<h3 style="font-family:Corbel Light; color:#737373">Fonctions</h3>

In [ ]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

<h3 style="font-family:Corbel Light; color:#737373">Extraction des features</h3>

In [ ]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

Ajout de l'analyse en composantes principales

In [ ]:
features_df_vec = features_df.select(col("path"),
                                     col("label"),
                                     array_to_vector('features').alias('features'))

In [ ]:
pca = PCA(k=31, inputCol="features", outputCol="pca_features")
pcaModel = pca.fit(features_df_vec) 

In [ ]:
explained_var = pcaModel.explainedVariance
cumValues = explained_var.cumsum()
print(cumValues)

In [ ]:
pca_results = pcaModel.transform(features_df_vec)

In [ ]:
pca_results.limit(5).show()

In [ ]:
reduced_features_df = pca_results.select(col("path"),
                                         col("label"),
                                         vector_to_array('pca_features').alias('features'))

In [ ]:
reduced_features_df.limit(5).show()

Enregistrement des données au format "parquet"

In [ ]:
reduced_features_df.write.mode("overwrite").parquet(PATH_Result)

<h2 style="font-family:Corbel Light; color:#737373">Chargement des données enregistrées et validation du résultat</h2>

In [ ]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

In [ ]:
df.head()

In [ ]:
df.loc[0,'features'].shape

In [ ]:
df.shape